In [0]:
# !pip install torch torchvision

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'rectina'

import matplotlib.pyplot as plt
import pandas as pd

ERROR:root:supported formats are: 'png','svg','pdf','png2x','jpg','retina','jpeg' not 'rectina'


In [37]:
#setting up the data

# !wget 'https://storage.googleapis.com/mledu-datasets/california_housing_train.csv'

df= pd.read_csv('california_housing_train.csv')
# df.loc[0]

np.array([df.iloc[6, 1:]]).astype('float')

array([[3.3610e+01, 2.5000e+01, 2.9070e+03, 6.8000e+02, 1.8410e+03,
        6.3300e+02, 2.6768e+00, 8.2400e+04]])

In [0]:
import numpy as np

class HousingDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.housing_frame = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.housing_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float')
        sample = {'landmarks': landmarks}

        return sample

In [0]:
train_size = int(0.8 * len(df))
test_size = len(df) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(df, [train_size, test_size])

In [0]:
#To normalize the data
transform = transforms.Compose([transforms.ToTensor(),
 transforms.Normalize((0.5,), (0.5,))
])

#Downloading and setting up the datas
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)